In [6]:
import pandas as pd
import numpy as np
from scipy.signal import find_peaks
import json

# 读取CSV文件
df = pd.read_csv('ecg_data.csv')

# 假设ECG数据在'ECG'列中
ecg_signal = df['ECG'].values

# 找到R波（R波是ECG信号中最明显的峰值）
r_peaks, _ = find_peaks(ecg_signal, height=0.5 * np.max(ecg_signal), distance=100)

# 找到P波和T波（P波在R波之前，T波在R波之后）
p_peaks = []
t_peaks = []

for r_peak in r_peaks:
    # P波在R波之前50-100个采样点
    p_window = ecg_signal[max(0, r_peak - 100):r_peak - 50]
    if len(p_window) > 0:
        p_peak = np.argmax(p_window) + max(0, r_peak - 100)
        p_peaks.append(p_peak)
    
    # T波在R波之后50-150个采样点
    t_window = ecg_signal[r_peak + 50:min(len(ecg_signal), r_peak + 150)]
    if len(t_window) > 0:
        t_peak = np.argmax(t_window) + r_peak + 50
        t_peaks.append(t_peak)

# 计算P波、R波和T波的振幅
p_amplitudes = ecg_signal[p_peaks]
r_amplitudes = ecg_signal[r_peaks]
t_amplitudes = ecg_signal[t_peaks]

# 计算平均振幅
avg_p_amplitude = np.mean(p_amplitudes)
avg_r_amplitude = np.mean(r_amplitudes)
avg_t_amplitude = np.mean(t_amplitudes)

# 根据条件判断情绪状态
if avg_p_amplitude < 0.25 and avg_t_amplitude > 0.1 * avg_r_amplitude:
    emotion_state = "0"   #正常
elif avg_p_amplitude > 0.25 and avg_t_amplitude > 0.1 * avg_r_amplitude:
    emotion_state = "1"   #情绪激动
elif avg_p_amplitude < 0.25 and avg_t_amplitude < 0.1 * avg_r_amplitude:
    emotion_state = "2"   #情绪焦虑抑郁等情感障碍
else:
    emotion_state = "3"   #无法判断

# 输出结果
print(f"平均P波振幅: {avg_p_amplitude:.2f} mV")
print(f"平均R波振幅: {avg_r_amplitude:.2f} mV")
print(f"平均T波振幅: {avg_t_amplitude:.2f} mV")
print(f"情绪状态: {emotion_state}")

result = {
    "avg_p_amplitude": avg_p_amplitude,
    "avg_r_amplitude": avg_r_amplitude,
    "avg_t_amplitude": avg_t_amplitude,
    "emotion_state": emotion_state,
}

with open("emotion_result.json", "w") as f:
    json.dump(result, f)

平均P波振幅: 555.90 mV
平均R波振幅: 1203.78 mV
平均T波振幅: 219.87 mV
情绪状态: 1
